In [1]:
# import dependencies 
import os
import gmaps
import pandas as pd

# import Google spi key from Config file
from config import gkey

# configure API key
gmaps.configure(api_key=gkey)

In [2]:
# create state with centroid DataFrame
povertyCSV = (os.path.join('povertyRateWithCentroids.csv'))
povertyDF = pd.read_csv(povertyCSV)
# remove index column that imported with CSV
del povertyDF['Unnamed: 0']
# preview poverty DataFrame
povertyDF.head()


,State,Latitude,Longitude,2012,2013,2014,2015,2016,2017
0,Alabama,32.7794,-86.8287,19.8,20.90,20.5,20.2,20.30,19.30
1,Alaska,64.0685,-152.2782,10.5,10.25,10.2,11.1,10.35,10.40
2,Arizona,34.2744,-111.6602,20.2,19.40,19.9,19.8,20.50,19.70
3,Arkansas,34.8938,-92.4426,20.4,21.50,21.1,21.0,20.60,19.80
4,California,37.1841,-119.4696,15.5,16.65,16.7,16.6,16.25,15.25


In [3]:
# store latitude and longitude values for state centroids
stateCentroid = povertyDF[['Latitude', 'Longitude']]

# store poverty %s for each state
povertyPercent2012 = povertyDF['2012'].astype(float)
povertyPercent2013 = povertyDF['2013'].astype(float)
povertyPercent2014 = povertyDF['2014'].astype(float)
povertyPercent2015 = povertyDF['2015'].astype(float)
povertyPercent2016 = povertyDF['2016'].astype(float)
povertyPercent2017 = povertyDF['2017'].astype(float)

In [4]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2012 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2012 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2012.add_layer(heat_layer)

# Display figure
fig2012

Figure(layout=FigureLayout(height='420px'))

In [5]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2013 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2013 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2013.add_layer(heat_layer)

# Display figure
fig2013

Figure(layout=FigureLayout(height='420px'))

In [6]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2014 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2014 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2014.add_layer(heat_layer)

# Display figure
fig2014

Figure(layout=FigureLayout(height='420px'))

In [7]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2015 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2015 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2015.add_layer(heat_layer)

# Display figure
fig2015

Figure(layout=FigureLayout(height='420px'))

In [8]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2016 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2016 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2016.add_layer(heat_layer)

# Display figure
fig2016

Figure(layout=FigureLayout(height='420px'))

In [9]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2017 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2017 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2017.add_layer(heat_layer)

# Display figure
fig2017

Figure(layout=FigureLayout(height='420px'))